In [47]:
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import pandas as pd
from fake_useragent import UserAgent
from bs4 import BeautifulSoup as bs
import urllib.request as ur
from get_risc_v_software_type import search_software_url
from tqdm import tqdm

In [48]:
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--headless")

url_and_repo_name = search_software_url()
url_and_repo_name

100%|███████████████████████████████████████| 22/22 [00:00<00:00, 265157.15it/s]


[['https://riscv.org/exchange/?_sft_exchange_category=software&_sfm_exchange_software_type=Accelerated%20Libraries',
  'Accelerated Libraries'],
 ['https://riscv.org/exchange/?_sft_exchange_category=software&_sfm_exchange_software_type=Accelerated%20Libraries,%20Linux,%20macOS',
  'Accelerated Libraries, Linux, macOS'],
 ['https://riscv.org/exchange/?_sft_exchange_category=software&_sfm_exchange_software_type=Application%20Infrastructure,%20Simulators',
  'Application Infrastructure, Simulators'],
 ['https://riscv.org/exchange/?_sft_exchange_category=software&_sfm_exchange_software_type=Bootloaders',
  'Bootloaders'],
 ['https://riscv.org/exchange/?_sft_exchange_category=software&_sfm_exchange_software_type=BSD%20Distro',
  'BSD Distro'],
 ['https://riscv.org/exchange/?_sft_exchange_category=software&_sfm_exchange_software_type=C%20Compilers%20and%20Libraries',
  'C Compilers and Libraries'],
 ['https://riscv.org/exchange/?_sft_exchange_category=software&_sfm_exchange_software_type=C%2

In [49]:
github_link = []
for url,repo_name in tqdm(url_and_repo_name):
    
    browser = webdriver.Chrome(options = chrome_options)
    browser.maximize_window()
    software_type_url = url

    browser.get(url)

    prev_height = browser.execute_script("return document.body.scrollHeight")

    # 웹페이지 맨 아래까지 무한 스크롤
    while True:
        # 스크롤을 화면 가장 아래로 내린다
        browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")

        # 페이지 로딩 대기
        time.sleep(2)

        # 현재 문서 높이를 가져와서 저장
        curr_height = browser.execute_script("return document.body.scrollHeight")

        if(curr_height == prev_height):
            break
        else:
            prev_height = browser.execute_script("return document.body.scrollHeight")



    html_source = browser.page_source
    soup = BeautifulSoup(html_source, 'lxml')


    
    software_list = soup.find_all("div", "col-results span_1_of_2-results bottom-results-right")
    
    for sw in software_list:

        link_name = sw.find('a')['href']
        if "github.com" in link_name:
            github_link.append(link_name)

100%|███████████████████████████████████████████| 22/22 [09:12<00:00, 25.13s/it]


In [50]:
github_link

['https://github.com/xianyi/OpenBLAS/tree/risc-v',
 'https://github.com/moepinet/libmoepgf.git',
 'https://github.com/olofk/vidbo',
 'https://github.com/riscv/opensbi',
 'https://github.com/riscv/riscv-pk',
 'https://github.com/luojia65/rustsbi',
 'https://github.com/riscv/riscv-uefi-edk2-docs',
 'https://github.com/freebsd/freebsd',
 'https://github.com/theandrew168/bronzebeard',
 'https://github.com/llvm/llvm-project',
 'https://github.com/AbsInt/CompCert.git',
 'https://github.com/riscv/riscv-gcc',
 'https://github.com/riscv/riscv-glibc',
 'https://github.com/riscv/riscv-musl',
 'https://github.com/riscv/riscv-newlib',
 'https://github.com/gnu-mcu-eclipse/openocd',
 'https://github.com/riscv/riscv-openocd',
 'https://github.com/eclipse/tcf.agent',
 'https://github.com/bao-project/bao-hypervisor',
 'https://github.com/kvm-riscv/linux.git',
 'https://github.com/kvm-riscv/kvmtool.git',
 'https://github.com/mit-pdos/rvirt',
 'https://github.com/xvisor/xvisor.git',
 'https://github.com/r

In [51]:
for link in github_link:
    
    df_dic = {}
    dic_keys = ['title','descript','topic_keyword','fork','star']
    
    url = link
    
    html = ur.urlopen(url)
    soup = bs(html.read(), "html.parser")
    
    repository_list = soup.find_all("body", "logged-in env-production page-responsive")
    
    for k in dic_keys:
        df_dic[k] = []
        
    for repo in repository_list:
        title = repo.find('a',{'data-pjax':"#repo-content-pjax-container"}).text.strip() # 각 repository별 title 추출

        description = repo.find('p','color-fg-muted mb-0 wb-break-word') # 각 repository별 drcript 추출
        if description is not None:
            descript = description.text.strip()
        else:
            descript = '[BLANK]'

        topic_keyword = repo.find('div','d-inline-flex flex-wrap flex-items-center f6 my-1') #각 repository별 topic keyword 추출

        if topic_keyword is not None:
            topic_keyword_list = []
            for tk in topic_keyword.find_all('a'):
                topic_keyword_list.append(tk.text.strip())
        else:
            topic_keyword_list = []

        get_fork = repo.find('a','Link--muted mr-3') #각 repository별 fork 수 추출

        if get_fork is not None:
            fork = int(get_fork.text.strip().replace(',',''))
        else:
            fork = 0

        get_star = repo.find('a','no-wrap Link--muted mr-3') #각 repository별 star 수 추출

        if get_star is not None:
            star = int(get_star.text.strip().replace(',',''))
        else:
            star = 0


        df_dic['title'].append(title)
        df_dic['descript'].append(descript)
        df_dic['topic_keyword'].append(topic_keyword_list)
        df_dic['fork'].append(fork)
        df_dic['star'].append(star)
    
    df = pd.DataFrame(
            columns=[
                    "title",
                    "descript",
                    "topic_keyword",
                    "fork",
                    "star",
        ]
    )
    

https://github.com/xianyi/OpenBLAS/tree/risc-v
https://github.com/moepinet/libmoepgf.git
https://github.com/olofk/vidbo
https://github.com/riscv/opensbi
https://github.com/riscv/riscv-pk
https://github.com/luojia65/rustsbi
https://github.com/riscv/riscv-uefi-edk2-docs
https://github.com/freebsd/freebsd
https://github.com/theandrew168/bronzebeard
https://github.com/llvm/llvm-project
https://github.com/AbsInt/CompCert.git
https://github.com/riscv/riscv-gcc
https://github.com/riscv/riscv-glibc
https://github.com/riscv/riscv-musl
https://github.com/riscv/riscv-newlib
https://github.com/gnu-mcu-eclipse/openocd
https://github.com/riscv/riscv-openocd
https://github.com/eclipse/tcf.agent
https://github.com/bao-project/bao-hypervisor
https://github.com/kvm-riscv/linux.git
https://github.com/kvm-riscv/kvmtool.git
https://github.com/mit-pdos/rvirt
https://github.com/xvisor/xvisor.git
https://github.com/riscv/riscv-gnu-toolchain
https://github.com/riscv/riscv-linux
https://github.com/riscv/meta-ri